### Consolidando vários arquivos .xlsx em um único DataFrame

##### OBSERVAÇÕES:

Ao setar o path dos arquivos é preciso ter prévio conhecimento das bases para que se consolide de forma correta e se há extenções diferente de planilha. 

*Exemplo: .xlsm*

In [10]:
#importação de bibliotecas para a automação e tratamento de dados
import os
import pandas as pd

In [11]:
#local dos arquivos
caminho = r'C:/Users/Lenovo/Desktop/PROJETOS_AUTOMACAO/CONSOLIDANDO_ARQUIVOS/BASE'
#setando diretorio dos arquivos
os.chdir(caminho)

In [12]:
#visualizando arquivos do diretório
arquivos =  os.listdir()
arquivos

['Pasta1.xlsx', 'Pasta2.xlsx', 'Pasta3.xlsx']

In [13]:
#lista  a ser populada
consolidado = []
#looping pegando todos os arquivos
for pasta in arquivos:
    if pasta.endswith('xlsx'): #filtro do tipo de arquivo por extensão
       consolidado.append(pd.read_excel(pasta))

tabela = pd.concat(consolidado, ignore_index= True , axis=0)

### Integração com SGBD e armazenamento dos dados

Para essa etapa é necessário que os dados tenham sido tratados anteriormente *(exemplo: filtros, tipagem, seleção de colunas)*

In [14]:
#importação da biblioteca para conexão com o banco de dados
import pyodbc
import pandas as pd

In [15]:
# Configurações do banco de dados Access
db_path = r'C:\Users\Lenovo\Desktop\PROJETOS_AUTOMACAO\CONSOLIDANDO_ARQUIVOS\BANCO_DADOS\BANCO_DE_DADOS.accdb'
driver = 'Microsoft Access Driver (*.mdb, *.accdb)'

# String de conexão
conn_str = f'DRIVER={{{driver}}};DBQ={db_path};'

# Conectar ao banco de dados
conn = pyodbc.connect(conn_str)
#onde será feito scripts sql
cursor = conn.cursor()

#limpando o banco de dados - uma vez que o diretório sempre terá os arquivos
consulta = cursor.execute('DELETE FROM tabela_teste WHERE Id >= 1')

In [16]:
#looping insert no banco de  dados
for lin in range(tabela.index.max() + 1):
    num = tabela.loc[lin][1]
    letras = tabela.loc[lin][0]

    #inserindo valores no BD
    insert_comando = f"""
    INSERT INTO tabela_teste(LETRA, NUMERO) 
    VALUES
    ('{letras}', {num})"""

    cursor.execute(insert_comando)
    cursor.commit()

In [17]:
#executando consulta SQL
consulta = cursor.execute('SELECT * FROM tabela_teste')

#lista - cabeçalho
colunas = [column[0] for column in cursor.description]
#lista - linhas
linhas = cursor.fetchall()
#transformando em tabela
tab_consulta = pd.DataFrame.from_records(linhas, columns=colunas)

In [18]:
#fechar a conexão com o BD
conn.close()

print("Dados inseridos no BD com sucesso!")

Dados inseridos no BD com sucesso!
